In [9]:
import pandas as pd
import os
import mdtraj as md
import numpy as np
from Bio import PDB
import dataclasses

from utils.pdbUtils import pdb_chain_parser, chain_str_to_int, write_pkl, parse_chain_feats, concat_np_features


In [3]:
pdb_dir = "raw_dnmt"
all_file_paths = [os.path.join(pdb_dir, x) for x in os.listdir(pdb_dir) if '.pdb' in x]
total_num_paths = len(all_file_paths)
print(total_num_paths)

61


In [4]:
write_dir = "test_protclip"
if not os.path.exists(write_dir):
    os.makedirs(write_dir)
metadata_file_name = 'metadata.csv'
metadata_path = os.path.join(write_dir, metadata_file_name)
print(f'Files will be written to {write_dir}')

Files will be written to test_protclip


In [21]:
for i, file_path in enumerate(all_file_paths):
    pdb_name = os.path.basename(file_path).replace('.pdb', '')
    print(pdb_name)
    parser = PDB.PDBParser(QUIET=True)
    structure = parser.get_structure(pdb_name, file_path)
    print(structure)
    struct_chains = {
        chain.id.upper(): chain
        for chain in structure.get_chains()}
    print(struct_chains)
    
    all_seqs = set()
    for chain_id, chain in struct_chains.items():
        # Convert chain id into int
        print(chain_id)
        chain_id = chain_str_to_int(chain_id)
        chain_prot = pdb_chain_parser(chain, chain_id)
        chain_dict = dataclasses.asdict(chain_prot)
        chain_dict = parse_chain_feats(chain_dict)
        all_seqs.add(tuple(chain_dict['aatype']))
        print(np.where(chain_prot.aatype != 20)[0])
        print(np.max(np.where(chain_prot.aatype != 20)[0]) - np.min(np.where(chain_prot.aatype != 20)[0]) + 1)
        
        #print(len(chain_prot.aatype))
    break

1g55
<Structure id=1g55>
{'A': <Chain id=A>}
A
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 209 210 211 212 213 214 215 216 217
 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235
 236

In [23]:
from Bio import PDB

def clip_protein(pdb_path, output_path, length=128):
    parser = PDB.PDBParser()
    structure = parser.get_structure('protein', pdb_path)

    for model in structure:
        for chain in model:
            # Create new structure for the clipped protein
            clipped_structure = PDB.Structure.Structure('clipped_protein')
            clipped_model = PDB.Model.Model(0)
            clipped_chain = PDB.Chain.Chain(chain.id)
            # Manually add residues up to the specified length
            for i, residue in enumerate(chain):
                if i >= length:
                    break  # Stop adding residues once we reach the desired length
                clipped_chain.add(residue)
            # Add the clipped chain to the model and the model to the structure
            clipped_model.add(clipped_chain)
            clipped_structure.add(clipped_model)
            # Save the clipped structure to a new PDB file
            io = PDB.PDBIO()
            io.set_structure(clipped_structure)
            io.save(output_path)


# Set file paths
pdb_file_path = 'raw_dnmt/1g55.pdb'
clipped_pdb_file_path = 'raw_dnmt/1g55_clipped.pdb'

# Clip the protein and save to a new PDB file
clip_protein(pdb_file_path, clipped_pdb_file_path)

clipped_pdb_file_path

'raw_dnmt/1g55_clipped.pdb'

In [25]:
for i, file_path in enumerate(all_file_paths):
    pdb_file_path = file_path
    pdb_name = os.path.basename(file_path).replace('.pdb', '')
    clipped_pdb_file_path = os.path.join("test_protclip", pdb_name+".pdb")
    clip_protein(pdb_file_path, clipped_pdb_file_path)

C:\Users\orkun\miniconda3\envs\ProteinFlow\lib\site-packages\Bio\PDB\Atom.py:232: PDBConstructionWarning: Used element 'U' for Atom (name=UNK) with given element 'X'
  warnings.warn(msg, PDBConstructionWarning)
C:\Users\orkun\miniconda3\envs\ProteinFlow\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 3621.
  warnings.warn(
C:\Users\orkun\miniconda3\envs\ProteinFlow\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 3647.
  warnings.warn(
C:\Users\orkun\miniconda3\envs\ProteinFlow\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 3660.
  warnings.warn(
C:\Users\orkun\miniconda3\envs\ProteinFlow\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 5525.
  warnings.warn(
C:\Users\orkun\miniconda3\envs\ProteinFlow\lib\site-packages\